Configurações para o notebook

In [1]:
import pandas as pd
import numpy as np

# Define a opção para mostrar TODAS as colunas, sem limite
pd.set_option('display.max_columns', None)

# Define a opção para mostrar TODAS as linhas, sem limite
pd.set_option('display.max_rows', None)


# Define a opção para mostrar o número máximo de elementos do array como infinito (None)
np.set_printoptions(threshold=np.inf)

Bibliotecas necessárias

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# LIMPANDO OS DADOS

Removendo colunas desncessárias no momento e registros de direções diferentes da direção NORTE

In [3]:
df_original = pd.read_csv('../dataset-completo.csv')

weakColumns   = ['B1_weak', 'B2_weak', 'B3_weak', 'B4_weak', 'B5_weak', 'B6_weak', 'B7_weak', 'B8_weak', 'B9_weak']
strongColumns = ['B1_strong', 'B2_strong', 'B3_strong', 'B4_strong', 'B5_strong', 'B6_strong', 'B7_strong', 'B8_strong', 'B9_strong']
avgColumns    = ['B1_avg', 'B2_avg', 'B3_avg', 'B4_avg', 'B5_avg', 'B6_avg', 'B7_avg',  'B8_avg', 'B9_avg']
delColum      = ['Alpha', 'Beta', 'Gamma', 'Seno', 'Cosseno', 'Created', 'Brand', 'Model']

df = df_original[df_original['Direction'] == 'norte']
dropColumns = weakColumns + strongColumns + delColum

df = df.drop(dropColumns, axis=1)
print(df.head())
print(df["Direction"].unique())

   B1_avg  B2_avg  B3_avg  B4_avg  B5_avg  B6_avg  B7_avg  B8_avg  B9_avg  \
0     -82     -82     -73    -105     -73     -66     -78     -72     -78   
1     -77     -85     -75     -76     -76     -66     -76     -76     -78   
2     -80     -84     -73     -74     -91     -68     -81     -74     -76   
3     -82     -83     -74     -77     -76     -68     -81     -72     -74   
4     -78     -85     -75     -77     -82     -66     -81     -75     -76   

  Direction     RP     X     Y Device  
0     norte  P_6_1  1.57  8.75  Luis3  
1     norte  P_6_1  1.57  8.75  Luis3  
2     norte  P_6_1  1.57  8.75  Luis3  
3     norte  P_6_1  1.57  8.75  Luis3  
4     norte  P_6_1  1.57  8.75  Luis3  
['norte']


Separando os dados de teste e treino do modelo, e teste simulando o ambiente real:

Dados de teste e treino do modelo -> Dados dos 9 beacons do dispositivo 'Moto7' com direção == 'Norte'

Dados teste (ambiente real) -> Dados dos 9 beacons dos demais dispositivos com direção == 'Norte'

In [4]:
devices = df["Device"].unique()
devices_sem_moto7 = [d for d in devices if d != "Moto7"]
print(devices_sem_moto7)


['Luis3', 'Wellington', 'Chaparro', 'Moto01', 'Moto6', 'Moto8', 'Joao', 'Josue', 'JulieS', 'Matteo', 'Francisco', 'Maggila', 'FelipeAmaral', 'Yuri', 'Moto2', 'Bruna', 'LuisHenrique', 'Horacio', 'Alexandre', 'fatima4', 'Jonathas', 'Moto5']


In [5]:
ponto = "P_6_3"
reference_device = "Moto7"

#Dispositivo de base sem o ponto de referência (RP!=ponto)
df_reference_device = df[(df["Device"] == reference_device) & (df["RP"] != ponto)]
print(df_reference_device.shape)

#Dados dos base no ponto de referência (RP=ponto)
df_reference_device_calibr = df[(df["Device"] == reference_device) & (df["RP"] == ponto)]
print(df_reference_device_calibr.shape)

#Dispositivo para teste (simulando ambiente real)
set_test_devices = {}
for device in devices_sem_moto7:
    set_test_devices[device] = df[(df["Device"] == device) & (df["RP"] != ponto)]

print(set_test_devices.keys())


#Dados dos dispositivos de teste no ponto de referência (RP=ponto)
set_test_devices_calibr = {}
for device in devices_sem_moto7:
    set_test_devices_calibr[device] = df[(df["Device"] == device) & (df["RP"] == ponto)]


print(set_test_devices_calibr["Yuri"])
print(set_test_devices_calibr["Yuri"].shape)


(754, 14)
(29, 14)
dict_keys(['Luis3', 'Wellington', 'Chaparro', 'Moto01', 'Moto6', 'Moto8', 'Joao', 'Josue', 'JulieS', 'Matteo', 'Francisco', 'Maggila', 'FelipeAmaral', 'Yuri', 'Moto2', 'Bruna', 'LuisHenrique', 'Horacio', 'Alexandre', 'fatima4', 'Jonathas', 'Moto5'])
       B1_avg  B2_avg  B3_avg  B4_avg  B5_avg  B6_avg  B7_avg  B8_avg  B9_avg  \
42930     -81     -72     -83     -86     -67     -74     -76     -78     -80   
42931     -81     -77     -84     -82     -73     -76     -80     -73     -78   
42932     -81     -79     -82     -80     -79     -83     -79     -72     -78   
42933     -81     -77     -79     -74     -73     -87     -77     -70     -77   
42934     -81     -78     -78     -73     -73     -84     -80     -70     -80   
42935     -83     -80     -78     -77     -76     -83     -79     -73     -79   
42936     -87     -76     -79     -76     -68     -79     -75     -75     -81   
42937     -86     -81     -79     -80     -63     -78     -80     -74     -79   
42

In [ ]:
RP = df_reference_device['RP']
SAMPLES_TRAIN = df_reference_device.drop(['RP', 'X', 'Y', 'Direction', 'Device'], axis=1)

X_train = SAMPLES_TRAIN
Y_train = RP


knn = KNeighborsClassifier(metric='euclidean', n_neighbors=1)
knn.fit(X_train, Y_train)

,n_neighbors,1
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'euclidean'
,metric_params,None
,n_jobs,None


# Predições + cálculo da calibragem

Cálculo dos valores de calibragem

In [7]:
def compute_calibration_bias(samples_base, samples_target):

    """
    Calcula o vetor de bias (fator de calibração) para alinhar o dispositivo target
    ao dispositivo base.
    """

    # Diferença ponto a ponto
    print(samples_base)
    print(samples_target)
    diffs = samples_base - samples_target # Matriz de diferenças

    # Mediana por coluna (por AP)
    bias = np.median(diffs, axis=0) 

    return bias

In [8]:
calibration_values = {}

for device, df_device in set_test_devices_calibr.items():

    print()
    print(f'Cálculo da calibração para o dispositivo: {device}')

    samples_base = df_reference_device_calibr.drop(['RP', 'X', 'Y', 'Direction', 'Device'], axis=1).to_numpy()
    samples_target = df_device.drop(['RP', 'X', 'Y', 'Direction', 'Device'], axis=1).to_numpy()
    
    min_rows = min(samples_base.shape[0], samples_target.shape[0])

    samples_base = samples_base[:min_rows]
    samples_target = samples_target[:min_rows]


    calibration_values[device] = compute_calibration_bias(samples_base, samples_target)
    print("Vetor de calibração (bias):", calibration_values[device])
    print()


Cálculo da calibração para o dispositivo: Luis3
[[-87 -81 -89 -84 -70 -85 -81 -75 -88]
 [-83 -81 -85 -80 -72 -82 -81 -77 -83]
 [-90 -82 -91 -81 -67 -86 -81 -74 -85]
 [-87 -82 -88 -82 -66 -86 -80 -73 -84]
 [-87 -83 -90 -81 -66 -89 -79 -74 -82]
 [-87 -86 -90 -83 -68 -90 -81 -74 -82]
 [-87 -85 -93 -84 -68 -87 -81 -73 -82]
 [-85 -88 -92 -84 -67 -87 -77 -76 -81]
 [-88 -86 -94 -83 -67 -85 -75 -74 -82]
 [-88 -86 -93 -83 -67 -84 -74 -73 -83]
 [-86 -86 -92 -83 -67 -88 -80 -73 -84]
 [-88 -89 -93 -82 -67 -88 -81 -73 -83]
 [-89 -89 -92 -85 -67 -89 -80 -74 -83]
 [-86 -93 -91 -83 -67 -91 -78 -73 -83]
 [-87 -90 -91 -85 -66 -89 -80 -74 -84]
 [-88 -88 -91 -87 -67 -90 -82 -73 -83]
 [-88 -90 -90 -86 -67 -92 -80 -72 -83]
 [-87 -85 -91 -84 -67 -87 -81 -73 -83]
 [-87 -87 -93 -84 -67 -90 -80 -72 -83]
 [-88 -88 -91 -84 -67 -90 -81 -73 -84]
 [-87 -88 -89 -88 -67 -91 -81 -72 -86]
 [-87 -85 -92 -85 -67 -91 -79 -72 -84]
 [-88 -86 -94 -84 -67 -87 -81 -73 -82]]
[[-105 -105  -80  -77  -74  -80  -74  -75  -80]
 [ -8

predição

In [51]:
for device, df_device in set_test_devices.items():
    print()
    print("Teste no dispositivo:", device)
    X_test = df_device.drop(['RP', 'X', 'Y', 'Direction', 'Device'], axis=1)
    Y_test = df_device['RP']


    predictions = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, predictions)
    print("Accuracy antes da calibração:", round(accuracy * 100,2), '%')

    
    X_test = X_test - calibration_values[device]
    predictions = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, predictions)
    print("Accuracy após calibração:", round(accuracy * 100,2), '%')
    print()


Teste no dispositivo: Luis3
Accuracy antes da calibração: 21.69 %
Accuracy após calibração: 17.04 %


Teste no dispositivo: Wellington
Accuracy antes da calibração: 25.56 %
Accuracy após calibração: 16.21 %


Teste no dispositivo: Chaparro
Accuracy antes da calibração: 26.39 %
Accuracy após calibração: 12.2 %


Teste no dispositivo: Moto01
Accuracy antes da calibração: 28.78 %
Accuracy após calibração: 16.84 %


Teste no dispositivo: Moto6
Accuracy antes da calibração: 44.83 %
Accuracy após calibração: 36.47 %


Teste no dispositivo: Moto8
Accuracy antes da calibração: 38.92 %
Accuracy após calibração: 28.9 %


Teste no dispositivo: Joao
Accuracy antes da calibração: 17.44 %
Accuracy após calibração: 5.33 %


Teste no dispositivo: Josue
Accuracy antes da calibração: 15.92 %
Accuracy após calibração: 7.03 %


Teste no dispositivo: JulieS
Accuracy antes da calibração: 16.41 %
Accuracy após calibração: 5.8 %


Teste no dispositivo: Matteo
Accuracy antes da calibração: 31.1 %
Accuracy apó